In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing import sequence
from get_data_rnn import load_data_by_num
from encoder_fcn_generate_data import load_fcn_encoding

Using TensorFlow backend.
/home/muhsin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# consensus network
X_1, y_1 = load_data_by_num(1)
X_2, y_2 = load_data_by_num(2)

X = np.vstack((X_1, X_2))
y = np.vstack((y_1, y_2))

y.shape, X.shape

((100, 6), (100, 8400))

In [3]:
# reshape such that [samples, time-step, features]
X = np.reshape(X, (X.shape[0], 100, 84))

X.shape
# 100 sample videos, for each with 100 frames (some padding), and each frame
# containing a vector of 120 features

(100, 100, 84)

In [4]:
# X_1, y_1 = load_fcn_encoding(1)
# X_2, y_2 = load_fcn_encoding(2)
# X_ = np.vstack((X_1, X_2))
# y_ = np.vstack((y_1, y_2))

# X_.shape, y_.shape

# X_, y = X_, y_

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [6]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((70, 100, 84), (70, 6), (30, 100, 84), (30, 6))

In [7]:
model = Sequential()
model.add(LSTM(100, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Dropout(0.2))
# model.add(LSTM(10, return_sequences=False))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               74000     
_________________________________________________________________
dense_1 (Dense)              (None, 2000)              202000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 2000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              2001000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               100100    
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
__________

In [9]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=10, verbose=2)

Train on 70 samples, validate on 30 samples
Epoch 1/50
1s - loss: 1.0677 - acc: 0.7857 - val_loss: 0.1406 - val_acc: 0.9667
Epoch 2/50
0s - loss: 0.0324 - acc: 1.0000 - val_loss: 1.3889e-04 - val_acc: 1.0000
Epoch 3/50
0s - loss: 4.2795e-04 - acc: 1.0000 - val_loss: 1.4484e-06 - val_acc: 1.0000
Epoch 4/50
0s - loss: 0.0012 - acc: 1.0000 - val_loss: 3.3180e-07 - val_acc: 1.0000
Epoch 5/50
0s - loss: 3.4332e-05 - acc: 1.0000 - val_loss: 3.2783e-07 - val_acc: 1.0000
Epoch 6/50
0s - loss: 0.0013 - acc: 1.0000 - val_loss: 2.3773e-05 - val_acc: 1.0000
Epoch 7/50
0s - loss: 0.0118 - acc: 0.9857 - val_loss: 1.4504e-07 - val_acc: 1.0000
Epoch 8/50
0s - loss: 0.0017 - acc: 1.0000 - val_loss: 2.4021e-06 - val_acc: 1.0000
Epoch 9/50
0s - loss: 3.7399e-04 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 10/50
0s - loss: 1.9693e-05 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 11/50
0s - loss: 1.2439e-05 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 12/50
0s

In [ ]:
import matplotlib.pyplot as plt
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'][:3])
plt.plot(history.history['val_acc'][:3])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'][:5])
plt.plot(history.history['val_loss'][:5])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [12]:
X_test.shape

(30, 100, 84)

In [13]:
for i in range(X_test.shape[0]):
    temp = np.reshape(X_test[i], (1, X_test[i].shape[0], X_test[i].shape[1]))
    temp_class = model.predict_classes(temp, verbose=0)[0]
    actual_class = y_test[i].argmax()
    truth = "*" if actual_class==temp_class else "/"
    print("{}\tprediction --> {}, actual --> {}".format(truth, temp_class, actual_class))

*	prediction --> 4, actual --> 4
*	prediction --> 0, actual --> 0
*	prediction --> 2, actual --> 2
*	prediction --> 5, actual --> 5
*	prediction --> 5, actual --> 5
*	prediction --> 4, actual --> 4
*	prediction --> 2, actual --> 2
*	prediction --> 4, actual --> 4
*	prediction --> 1, actual --> 1
*	prediction --> 0, actual --> 0
*	prediction --> 1, actual --> 1
*	prediction --> 4, actual --> 4
*	prediction --> 2, actual --> 2
*	prediction --> 4, actual --> 4
*	prediction --> 5, actual --> 5
*	prediction --> 0, actual --> 0
*	prediction --> 2, actual --> 2
*	prediction --> 2, actual --> 2
*	prediction --> 1, actual --> 1
*	prediction --> 4, actual --> 4
*	prediction --> 0, actual --> 0
*	prediction --> 4, actual --> 4
*	prediction --> 2, actual --> 2
*	prediction --> 5, actual --> 5
*	prediction --> 1, actual --> 1
*	prediction --> 1, actual --> 1
*	prediction --> 5, actual --> 5
*	prediction --> 5, actual --> 5
*	prediction --> 0, actual --> 0
*	prediction --> 2, actual --> 2


In [16]:
for i in range(X.shape[0]):
    temp = np.reshape(X[i], (1, X[i].shape[0], X[i].shape[1]))
    temp_class = model.predict_classes(temp, verbose=0)[0]
    actual_class = y[i].argmax()
    truth = "%" if actual_class==temp_class else "\\"
    print("{}\tprediction --> {}, actual --> {}".format(truth, temp_class, actual_class))

%	prediction --> 0, actual --> 0
%	prediction --> 0, actual --> 0
%	prediction --> 0, actual --> 0
%	prediction --> 0, actual --> 0
%	prediction --> 0, actual --> 0
%	prediction --> 0, actual --> 0
%	prediction --> 0, actual --> 0
%	prediction --> 0, actual --> 0
%	prediction --> 0, actual --> 0
%	prediction --> 0, actual --> 0
%	prediction --> 1, actual --> 1
%	prediction --> 1, actual --> 1
%	prediction --> 1, actual --> 1
%	prediction --> 1, actual --> 1
%	prediction --> 1, actual --> 1
%	prediction --> 1, actual --> 1
%	prediction --> 1, actual --> 1
%	prediction --> 1, actual --> 1
%	prediction --> 1, actual --> 1
%	prediction --> 1, actual --> 1
%	prediction --> 2, actual --> 2
%	prediction --> 2, actual --> 2
%	prediction --> 2, actual --> 2
%	prediction --> 2, actual --> 2
%	prediction --> 2, actual --> 2
%	prediction --> 2, actual --> 2
%	prediction --> 2, actual --> 2
%	prediction --> 2, actual --> 2
%	prediction --> 2, actual --> 2
%	prediction --> 2, actual --> 2
%	predicti

In [17]:
class_num = []
for y_c in y_test:
    class_num.append(y_c.argmax())

In [18]:
for i in range(6):
    print(i, class_num.count(i))

0 5
1 5
2 7
3 0
4 7
5 6


In [19]:
# Final evaluation of the model
scores = model.evaluate(X, y, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

100/100 [==============================] - 0s     
Accuracy: 100.00%


In [20]:
tester = np.reshape(X_test[0], (1, X_test[0].shape[0], X_test[0].shape[1]))
model.predict(tester)

array([[  1.88780675e-12,   3.76296134e-16,   2.84664925e-15,
          1.46542073e-20,   1.00000000e+00,   2.19654958e-12]], dtype=float32)

In [21]:
model.predict_classes(tester)

1/1 [==============================] - 0s


array([4])

In [22]:
dir_to_save = './rnn_model/model_fc2_main.h5'

In [23]:
model.save(dir_to_save)

In [24]:
# type two model

In [26]:
## test from saved model!

from keras.models import load_model
# dir_to_save = './rnn_model/model_fcn_net.h5'
dir_to_save = './rnn_model/model_fc2_main.h5'
model = load_model(dir_to_save)
model.summary()

X_1, y_1 = load_fcn_encoding(1)
X_2, y_2 = load_fcn_encoding(2)
X_ = np.vstack((X_1, X_2))
y_ = np.vstack((y_1, y_2))

# Final evaluation of the model
scores = model.evaluate(X_, y_, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               74000     
_________________________________________________________________
dense_1 (Dense)              (None, 2000)              202000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 2000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              2001000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               100100    
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
__________

ValueError: Error when checking input: expected lstm_1_input to have shape (None, 100, 84) but got array with shape (100, 100, 160)